In [1]:
# For Ubuntu 22.04 DCAP Attestation (quote generation and quote verification) in ACC
# Intel® SGX Data Center Attestation Primitives

In [ ]:
sudo apt-get update
sudo apt-get install build-essential

In [ ]:
#Download and installation of SGX DCAP repo (including the quote verification and generation)
wget https://download.01.org/intel-sgx/latest/dcap-latest/linux/distro/ubuntu22.04-server/sgx_linux_x64_sdk_2.25.100.3.bin ; chmod +x sgx*.bin ; sudo ./sgx_linux_x64_sdk_2.25.100.3.bin --prefix /opt/intel

echo 'deb [signed-by=/etc/apt/keyrings/intel-sgx-keyring.asc arch=amd64] https://download.01.org/intel-sgx/sgx_repo/ubuntu jammy main' | sudo tee /etc/apt/sources.list.d/intel-sgx.list ; wget https://download.01.org/intel-sgx/sgx_repo/ubuntu/intel-sgx-deb.key ; sudo mkdir /etc/apt/keyrings ; cat intel-sgx-deb.key | sudo tee /etc/apt/keyrings/intel-sgx-keyring.asc > /dev/null ; sudo apt-get update

sudo apt-get install -y libsgx-urts libsgx-launch libsgx-epid libsgx-quote-ex libsgx-enclave-common-dev libsgx-uae-service libsgx-dcap-default-qpl libsgx-dcap-default-qpl-dev libsgx-quote-ex-dev libsgx-dcap-ql-dev libsgx-dcap-ql libsgx-dcap-quote-verify libsgx-dcap-quote-verify-dev


In [ ]:
#adding user to launch an enclave
sudo usermod -aG sgx $USER # add user to the "sgx" user group
sudo usermod -aG sgx_prv $USER # add user to the "sgx_prv" user group which allows "in-process" quote generation, i.e., it loads provisioning/quoting enclaves by itself

### At this point, please restart ssh session/shell upon successfull setting of user privlleges

In [ ]:
# Repointing the SGX Attestation to fetch security baseline from Azure's Trusted Hardware Identity Management (THIM)

cp /etc/sgx_default_qcnl.conf ~

wget https://raw.githubusercontent.com/intel/SGXDataCenterAttestationPrimitives/master/QuoteGeneration/qcnl/linux/sgx_default_qcnl_azure.conf

sudo cp sgx_default_qcnl_azure.conf /etc/sgx_default_qcnl.conf

sudo vi /etc/sgx_default_qcnl.conf  # Just to confirm pointing to Azure THIM service


In [ ]:
# Clone/Download the SGX Data Center Attestation primitives to use Quote Generation / Verification
git clone https://github.com/intel/SGXDataCenterAttestationPrimitives.git

In [ ]:
#Build using make and generate a quote key
cd SGXDataCenterAttestationPrimitives/SampleCode/QuoteGenerationSample

make

./app

The following will be outputed
```
	### Successful Output of Quote Generation - quote with cert_key_type = 0x5 is created
	[APP] Step1: Call sgx_qe_get_target_info:
	[APP] succeed!
	[APP] Step2: Call create_app_report
	[APP] Step3: Call sgx_qe_get_quote_size
	[APP] Step4: Call sgx_qe_get_quote
	[APP] cert_key_type = 0x5

```

In [ ]:
cd ../QuoteVerificationSample

make SGX_DEBUG=1  # Add QPL_LOGGING=2 for lots more logging to the screen

./app

```
	### Successful Output of Quote Verification - 0xa007 result == SGX_QL_QV_RESULT_SW_HARDENING_NEEDED, as expected
	[APP] Info: ECDSA quote path: ../QuoteGenerationSample/quote.dat
	[APP] Trusted quote verification:
	[APP] Info: get target info successfully returned.
	[APP] Info: sgx_qv_set_enclave_load_policy successfully returned.
	[APP] Info: tee_get_quote_supplemental_data_version_and_size successfully returned.
	[APP] Info: latest supplemental data major version: 3, minor version: 3, size: 536
	[APP] Info: App: tee_verify_quote successfully returned.
	[APP] Info: Ecall: Verify QvE report and identity successfully returned.
	[APP] Warning: App: Verification completed with Non-terminal result: a007
	[APP] Info: Supplemental data Major Version: 3
	[APP] Info: Supplemental data Minor Version: 3
	[APP] Info: Advisory ID: INTEL-SA-00615

	===========================================

	[APP] Untrusted quote verification:
	[APP] Info: tee_get_quote_supplemental_data_version_and_size successfully returned.
	[APP] Info: latest supplemental data major version: 3, minor version: 3, size: 536
	[APP] Info: App: tee_verify_quote successfully returned.
	[APP] Warning: App: Verification completed with Non-terminal result: a007
	[APP] Info: Supplemental data Major Version: 3
	[APP] Info: Supplemental data Minor Version: 3
	[APP] Info: Advisory ID: INTEL-SA-00615
  
```

For 0xa007 result code (and others), see this file:  Result codes here:  https://github.com/intel/SGXDataCenterAttestationPrimitives/blob/2562057f6a3149c03f5985826ffaba978ece58c2/QuoteVerification/QvE/Include/sgx_qve_header.h#L60